<a href="https://colab.research.google.com/github/Bhairavi-shah/AbstractiveUserReviewConsolidation/blob/master/Module_2_%7C_Direct_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%time
from google.colab import drive
drive.mount('/content/gdrive')

import pandas as pd
pd.set_option('display.max_colwidth',1000)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
CPU times: user 415 ms, sys: 109 ms, total: 524 ms
Wall time: 52.7 s


In [2]:
df = pd.read_json(r"/content/gdrive/My Drive/PROJECT | S7-S8/Data/AMAZON_FASHION_5.json", lines = True)
df.groupby('asin').mean()
df_test = (df.loc[df['asin'] == 'B0014F7B98'])
print(df_test)
print(df_test.columns)

      overall  verified   reviewTime  ... unixReviewTime  vote image
869         5      True  03 20, 2016  ...     1458432000   NaN   NaN
870         5      True  03 18, 2016  ...     1458259200   NaN   NaN
871         4      True  03 16, 2016  ...     1458086400   NaN   NaN
872         5      True  03 13, 2016  ...     1457827200  22.0   NaN
873         5      True   03 6, 2016  ...     1457222400   NaN   NaN
...       ...       ...          ...  ...            ...   ...   ...
3146        5      True  05 22, 2018  ...     1526947200   NaN   NaN
3147        5      True  05 16, 2018  ...     1526428800   NaN   NaN
3148        5      True  05 16, 2018  ...     1526428800   NaN   NaN
3149        3      True  05 15, 2018  ...     1526342400   NaN   NaN
3150        3      True  05 15, 2018  ...     1526342400   NaN   NaN

[374 rows x 12 columns]
Index(['overall', 'verified', 'reviewTime', 'reviewerID', 'asin', 'style',
       'reviewerName', 'reviewText', 'summary', 'unixReviewTime', 'vote'

In [49]:
import nltk
import pickle
import re, string, random
from nltk.tag import pos_tag
from google.colab import files
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import twitter_samples, stopwords
from nltk import FreqDist, classify, NaiveBayesClassifier

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('twitter_samples')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [51]:
def remove_noise(tweet_tokens, stop_words = ()):
  cleaned_tokens = []
  for token, tag in pos_tag(tweet_tokens):
    token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
    token = re.sub("(@[A-Za-z0-9_]+)","", token)
    if token in Abbr_dict.keys():
      token = Abbr_dict[token]
    if tag.startswith("NN"):
      pos = 'n'
    elif tag.startswith('VB'):
      pos = 'v'
    else:
      pos = 'a'
    lemmatizer = WordNetLemmatizer()
    token = lemmatizer.lemmatize(token, pos)
    if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
      cleaned_tokens.append(token.lower())
  return cleaned_tokens


def get_all_words(cleaned_tokens_list):
  for tokens in cleaned_tokens_list:
    for token in tokens:
      yield token

def get_tweets_for_model(cleaned_tokens_list):
  for tweet_tokens in cleaned_tokens_list:
    yield dict([token, True] for token in tweet_tokens)

positive_tweets = twitter_samples.strings('positive_tweets.json')
negative_tweets = twitter_samples.strings('negative_tweets.json')
text = twitter_samples.strings('tweets.20150430-223406.json')

stop_words = stopwords.words('english')

positive_tweet_tokens = twitter_samples.tokenized('positive_tweets.json')
negative_tweet_tokens = twitter_samples.tokenized('negative_tweets.json')

positive_cleaned_tokens_list = []
negative_cleaned_tokens_list = []

for tokens in positive_tweet_tokens:
  positive_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

for tokens in negative_tweet_tokens:
  negative_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

all_pos_words = get_all_words(positive_cleaned_tokens_list)

freq_dist_pos = FreqDist(all_pos_words)
print(freq_dist_pos.most_common(10))

positive_tokens_for_model = get_tweets_for_model(positive_cleaned_tokens_list)
negative_tokens_for_model = get_tweets_for_model(negative_cleaned_tokens_list)

positive_dataset = [(tweet_dict, "Positive") for tweet_dict in positive_tokens_for_model]

negative_dataset = [(tweet_dict, "Negative") for tweet_dict in negative_tokens_for_model]

dataset = positive_dataset + negative_dataset

random.shuffle(dataset)

train_data = dataset[:7000]
test_data = dataset[7000:]

classifier = NaiveBayesClassifier.train(train_data)
print("Score Naive Bayes:", classify.accuracy(classifier, test_data))

pickle.dump(classifier, open("nb.pkl", 'wb'))
files.download('nb.pkl')

[(':)', 3691), (':-)', 701), (':d', 658), ('thanks', 391), ('follow', 357), ('love', 337), ('...', 290), ('good', 286), ('get', 263), ('thank', 253)]
Score Naive Bayes: 0.9976666666666667


In [52]:
for i, row in df_test.iterrows():
  t = row['reviewText']
  ts = remove_noise(word_tokenize(str(t)))
  print(t, classifier.classify(dict([tok, True] for tok in ts)))

Put them on and walked 3 hours with no problem! Love them! So light feeling Positive
excelente Positive
The shoes fit well in the arch area. They are a little wider in the toe area of the shoe, you feel like you have a lot of room. This does not make the shoe uncomfortable, just had to get used to it. Love the shoe. Negative
Tried them on in a store before buying online so I knew they'd fit good. Overall I was looking for a durable cross training shoe that would hold up to my rigorous training and these have been great so far. They are really light and comfortable. Most importantly for me they have grips on the bottoms so my feet don't slide out from under me while doing planks, push-ups, etc. Highly satisfied with this purchase. Negative
I recommend that! Positive
My son likes these, and this is the 2nd pair he's worn. Positive
Really great for walking I'm very glad I got these and the color is fun Positive
Love these shoes. My feet feel so much better. Lots of padding and light enoug